In [ ]:
include("preload.jl")
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "")
@pyimport python2juliaReuse as p2j
include("preload.jl")
pc = p2j.PolicyContainer()
using Distributions

In [ ]:
pc[:reset_policy](policy_path="Data/Normal/Policy/fsp_4e3.pkl")

In [ ]:
function handle_action!(scene,models,roadway,text;reverse=false)
    state = get_observation(scene,models,roadway)

    action1_arr = pc[:getAction](state,1)
    action1_arr = tanh.(action1_arr)
    
#     dacc = rand(pareto)-2.0
#     dsteer = rand(pareto)-2.0
#     if reverse
#         dacc = -dacc
#         dsteer = -dsteer
#     end
    dacc = 0.0
    dsteer = 0.0
    
    action = Egoaction(action1_arr[1],action1_arr[2],dacc,dsteer)
    bump,fail = simulate_action!(action,scene,models,roadway)
    reward = reward_fn(action,fail,scene,models,roadway)
    if bump
        rand_ego!(scene,models,roadway)
        pc[:resetPolicy](1)
    end
    state = get_observation(scene,models,roadway)
    empty!(text)
    push!(text,string("speed: ",scene[1].state.v))
    render(scene,roadway,models,text=text,cam=CarFollowCamera{Int}(1, 10.0))
end

In [ ]:
function drawsim(t,dt=NaN)
    handle_action!(scene,models,roadway,text,reverse=true)
end

In [ ]:
scene, models, roadway = initialize_env()
text = ["Nothing"]
pc[:resetPolicy](1)
pareto=Pareto(1.0,1)
rand_ego!(scene,models,roadway)
render(scene,roadway)

In [ ]:
drawsim(0.0)

In [ ]:
framerate = 20
ticks = fps(framerate)
timestamps = map(_ -> time(), ticks)
map(drawsim,timestamps)

In [ ]:
using Reel
framerate = 24

film = roll(drawsim, fps=framerate, duration=30)
write("Data/White/TanhEnv/Data/GIF/fsp15e3_pareto1_reverse.gif", film) # Write to a gif file